<a href="https://colab.research.google.com/github/Hamza-Chekireb/Artificial-Intelligence-Data-Science-Data-Analysis-Portfolio/blob/main/Sentiment_analysis_emojifier_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**--Atelier 13 octobre 2022 et 21 octobre 2022**

**--Text classifier**

In [ ]:
import numpy as np
import pandas as pd
! pip install emoji
import emoji
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding

In [ ]:
data = pd.read_csv('/Users/Hamza/Downloads/Emojify-Text-main/emoji_data.csv', header = None)

In [ ]:
#Comment ça marche l'emojize
emoji.emojize(':fork_and_knife_with_plate:')

# Créer uun dictionnaire pour assigner un emoji à chaque label
emoji_dict = {0: ":red_heart:",    
                    1: ":baseball:",
                    2: ":grinning_face_with_big_eyes:",
                    3: ":disappointed_face:",
                    4: ":fork_and_knife_with_plate:"}

# Créer une fonction de conversion de chiffre en emoji

def label_to_emoji(label):
  return emoji.emojize(emoji_dict[label])


In [ ]:
# Visualiser les valeurs uniques des Commentaires et les valeurs uniques des étiquettes
x = data[0].values
y = data[1].values
print(x)
print(y)


In [ ]:
#np.unique(y)
data[1].unique()

data[1].replace({ '0 ': '0', '1 ': '1', '2 ': '2','3 ': '3', '4 ': '4','0v2':'0'}, inplace=True)
print(data[1].unique())


In [ ]:
file = open('/Users/Hamza/Downloads/glove.6B.50d.txt','r', encoding = 'utf8') # Ouvrir le text qui contient Embedding
content = file.readlines()
file.close()
content

In [ ]:
# Un Word_Embedding pré_entraîné 

embeddings = {}
for line in content :
    # Convertir le Embedding des mots en une liste avec des valeurs de vecteur
    line = line.split()# Tokens 
    # Mettre le premier mot en tant que clé et le reste en tant que valeurs Converties en tableau numpy de valeurs numériques 
    embeddings[line[0]] = np.array(line[1:], dtype = float)
    
embeddings    
    
    

In [ ]:
from keras.preprocessing.text import Tokenizer 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

# Créer le vocabulaire avec les index
word2index = tokenizer.word_index
word2index

In [ ]:
#Convertir tous les commentaire en séquence d'index
xtokens = tokenizer.texts_to_sequences(x)
xtokens

In [ ]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen,len(sent))
    return maxlen

maxlen = get_maxlen(xtokens)

In [ ]:
# Remplir les séquences par des zéros pour qu'ils aient la même longueur 
from keras.preprocessing.sequence import pad_sequences 
xtrain = pad_sequences(xtokens, maxlen = maxlen, padding = 'post', truncating='post')
xtrain

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
ytrain = to_categorical(y)
ytrain

In [ ]:
embed_size = 50
embedding_matrix = np.zeros((len(word2index)+1, embed_size)) # len(word2index)+1 : car le premier indice de word2index est 1
embedding_matrix
for word,i in word2index.items():
    embed_vector = embeddings[word] #embeddings: Embedding_Words préentraîné
    embedding_matrix[i] = embed_vector

print(embedding_matrix)
print(embedding_matrix.shape)

In [ ]:
model = Sequential([
    # input_dim : La taille du vocabulaire+1
    # output_dim : La taille des vecteurs dans le Word_Embedding utilisé
    # input_len : la taille des séquences
    # weights : C'est une matrice avec le Word_Embedding de tous les mots du vocabulaire
    Embedding(input_dim = len(word2index)+1, output_dim = 50, input_length = maxlen, weights = [embedding_matrix],trainable = False),
    SimpleRNN(units=16),
    Dense(5, activation= 'softmax')
    
    
])



In [ ]:
import tensorflow
model.compile(optimizer='adam', loss = tensorflow.keras.metrics.categorical_crossentropy, metrics = ['accuracy'])
model.fit(xtrain,ytrain,epochs = 100)

**Tester le modèle**

In [ ]:
test = ['happy men', "i play baseball", 'i want to eat']
testseq = tokenizer.texts_to_sequences(test)
print(testseq)

xtest= pad_sequences(testseq, maxlen = maxlen, padding = 'post')
xtest

In [ ]:
ypred = model.predict(xtest)
ypred
ypred = np.argmax(ypred,axis = 1)
ypred

In [ ]:
for i in range(len(test)):
    print(test[i],label_to_emoji(ypred[i]))